# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [2]:
%run update_path.py

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
# Write your code below.
from glob import glob
import os
import pandas as pd
import random
import dask.dataframe as dd
import dask

In [20]:
#os.getenv('PRICE_DATA')
#glob(os.path.join(os.getenv('PRICE_DATA')))
#PRICE_DATA = os.getenv("PRICE_DATA")

In [4]:
# glob(os.path.join(os.getenv('PRICE_DATA'), "**", "*.parquet"), recursive=True
parquet_files = glob(os.path.join(os.getenv('PRICE_DATA'), "**", "*.parquet"), recursive=True)
parquet_files

['../../05_src/data/prices\\ABT\\ABT_1980\\part.0.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1980\\part.1.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1981\\part.0.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1981\\part.1.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1982\\part.0.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1982\\part.1.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1983\\part.0.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1983\\part.1.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1984\\part.0.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1984\\part.1.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1985\\part.0.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1985\\part.1.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1986\\part.0.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1986\\part.1.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1987\\part.0.parquet',
 '../../05_src/data/prices\\ABT\\ABT_1987\\part.1.parquet',
 '../../05_src/data/prices\\ABT\\ABT_198

In [22]:
#random.seed(42)
#n_sample = 60
#parquet_files = random.sample(parquet_files, n_sample)

In [5]:
# Read  Dask data 
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
#dd_price = dd.read_parquet(parquet_files)
dd_px

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
npartitions=90,,,,,,,,,
ABT,datetime64[ns],float64,float64,float64,float64,float64,int64,string,int32
ACN,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...


In [24]:
#feature- using th e assign not support the meta data:
#dd_feat = dd_price.assign(         # 
 #   Close_lag_1=dd_price.groupby("ticker")["Close"].shift(1),
  #  Adj_Close_lag_1=dd_price.groupby("ticker")["Adj Close"].shift(1),
   # returns=lambda df: (df["Close"] / df.groupby("ticker")["Close"].shift(1)) - 1, #shift(1) moves the Close value one day back
    #hi_lo_range=lambda df: df["High"] - df["Low"] #high and low price of ticker
#)

In [6]:
#feature- using the groupby:

dd_feat = (                                                                      # Assign the result to dd_feat
    dd_px
        .groupby("ticker", group_keys=False)
        .apply(
            lambda x: (
                x.sort_values("Date")
                 .assign(
                     Close_lag_1=x["Close"].shift(1),                           # Add lags for variables Close and Adj_Close.
                     Adj_Close_lag_1=x["Adj Close"].shift(1), 
                     returns=lambda df: (df["Close"] / df["Close_lag_1"]) - 1,  # Add returns based on Close: returns: (Close / Close_lag_1) - 1
                     hi_lo_range=lambda df: df["High"] - df["Low"]              # Add the following range: hi_lo_range: this is the day's High minus Low.
                 )
            ),
            meta=pd.DataFrame(
                data={
                    "Date": "datetime64[ns]",
                    "Open": "f8",
                    "High": "f8",
                    "Low": "f8",
                    "Close": "f8",
                    "Adj Close": "f8",
                    "Volume": "i8",
                    "source": "object",
                    "Year": "int32",
                    "Close_lag_1": "f8",
                    "Adj_Close_lag_1": "f8",
                    "returns": "f8",
                    "hi_lo_range": "f8",
                },
                index=pd.Index([], dtype=pd.StringDtype(), name="ticker")
            )
        )
)


In [7]:
dd_feat 


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=90,,,,,,,,,,,,,
ABT,object,object,object,object,object,object,object,object,object,object,object,object,object
ACN,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [8]:
# Compute the data and convert the panadas data frame
df = dd_feat.compute()
df

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
ABT,1980-03-17,0.00,0.512028,0.497999,0.501506,0.009856,7513300.0,ABT.csv,1980,NaN,NaN,NaN,0.014028
ABT,1980-03-18,0.00,0.512028,0.494492,0.505013,0.009925,5303400.0,ABT.csv,1980,0.501506,0.009856,0.006993,0.017535
ABT,1980-03-19,0.00,0.515535,0.503260,0.512028,0.010062,2523300.0,ABT.csv,1980,0.505013,0.009925,0.013889,0.012275
ABT,1980-03-20,0.00,0.513781,0.499753,0.499753,0.009821,4654800.0,ABT.csv,1980,0.512028,0.010062,-0.023973,0.014028
ABT,1980-03-21,0.00,0.506767,0.499753,0.505013,0.009925,1332800.0,ABT.csv,1980,0.499753,0.009821,0.010526,0.007014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.06,4.530000,3.880000,4.510000,4.510000,1668500.0,ZIXI.csv,2020,4.000000,4.000000,0.127500,0.650000
ZIXI,2020-03-27,4.49,4.710000,4.100000,4.600000,4.600000,1146800.0,ZIXI.csv,2020,4.510000,4.510000,0.019956,0.610000
ZIXI,2020-03-30,4.83,4.870000,4.440000,4.640000,4.640000,1212000.0,ZIXI.csv,2020,4.600000,4.600000,0.008696,0.430000


In [9]:
#df.groupby(level='ticker')['returns'].mean()
df.groupby('ticker')['returns'].mean()


ticker
ABT     0.000629
ACN     1.061834
ACY     0.486658
ALDX    0.000499
ALL     0.488665
          ...   
VIAC    0.312073
WORK   -0.001129
WPG    -0.001917
WST     4.130909
ZIXI    0.012603
Name: returns, Length: 90, dtype: float64

In [10]:
#  10-day moving average of returns per ticker in pandas data frame
df["returns_movingaver"] = (
    df
    .groupby("ticker")["returns"]
    .rolling(10)
    .mean()
    .reset_index(level=0, drop=True)
)

In [30]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_movingaver
ticker,,,,,,,,,,,,,,
ABT,1980-03-17,0.00,0.512028,0.497999,0.501506,0.009856,7513300.0,ABT.csv,1980,NaN,NaN,NaN,0.014028,NaN
ABT,1980-03-18,0.00,0.512028,0.494492,0.505013,0.009925,5303400.0,ABT.csv,1980,22.944036,17.157808,-0.977989,0.017535,NaN
ABT,1980-03-19,0.00,0.515535,0.503260,0.512028,0.010062,2523300.0,ABT.csv,1980,23.159946,17.319271,-0.977892,0.012275,NaN
ABT,1980-03-20,0.00,0.513781,0.499753,0.499753,0.009821,4654800.0,ABT.csv,1980,23.159946,17.319271,-0.978422,0.014028,NaN
ABT,1980-03-21,0.00,0.506767,0.499753,0.505013,0.009925,1332800.0,ABT.csv,1980,23.111965,17.283386,-0.978149,0.007014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.06,4.530000,3.880000,4.510000,4.510000,1668500.0,ZIXI.csv,2020,7.533333,7.382380,-0.401327,0.650000,-0.488082
ZIXI,2020-03-27,4.49,4.710000,4.100000,4.600000,4.600000,1146800.0,ZIXI.csv,2020,7.533333,7.382380,-0.389381,0.610000,-0.489807
ZIXI,2020-03-30,4.83,4.870000,4.440000,4.640000,4.640000,1212000.0,ZIXI.csv,2020,7.266667,7.121057,-0.361468,0.430000,-0.482237


In [47]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_movingaver
ticker,,,,,,,,,,,,,,
ABT,1980-03-17,0.00,0.512028,0.497999,0.501506,0.009856,7513300.0,ABT.csv,1980,NaN,NaN,NaN,0.014028,NaN
ABT,1980-03-18,0.00,0.512028,0.494492,0.505013,0.009925,5303400.0,ABT.csv,1980,22.944036,17.157808,-0.977989,0.017535,NaN
ABT,1980-03-19,0.00,0.515535,0.503260,0.512028,0.010062,2523300.0,ABT.csv,1980,23.159946,17.319271,-0.977892,0.012275,NaN
ABT,1980-03-20,0.00,0.513781,0.499753,0.499753,0.009821,4654800.0,ABT.csv,1980,23.159946,17.319271,-0.978422,0.014028,NaN
ABT,1980-03-21,0.00,0.506767,0.499753,0.505013,0.009925,1332800.0,ABT.csv,1980,23.111965,17.283386,-0.978149,0.007014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.06,4.530000,3.880000,4.510000,4.510000,1668500.0,ZIXI.csv,2020,4.000000,4.000000,0.127500,0.650000,-0.000807
ZIXI,2020-03-27,4.49,4.710000,4.100000,4.600000,4.600000,1146800.0,ZIXI.csv,2020,4.510000,4.510000,0.019956,0.610000,0.004262
ZIXI,2020-03-30,4.83,4.870000,4.440000,4.640000,4.640000,1212000.0,ZIXI.csv,2020,4.600000,4.600000,0.008696,0.430000,0.015491


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

Answer:

Pandas conversion is convenient but not required. Since Dask supports rolling window operations (such as groupby().rolling().mean()),
it is possible to compute the moving average of returns directly in Dask without converting to pandas. Dask is only “better” when 
the dataset is very large (larger than available RAM) and requires distributed or out-of-core processing to avoid memory issues. 

Dask uses lazy evaluation and builds a task graph, executing computations only when needed.For large datasets, it is better to use Dask 
because it can handle data that does not fit entirely in memory by processing it in 
partitions. 

Converting to pandas requires loading all the data into memory, which can be slow or even impossible for large datasets. 
However, pandas is simpler and more convenient for smaller datasets or for the final steps of analysis.



Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.